In [ ]:
# 检测ollama是否正在运行
# 运行ollama: ollama run llama3
import psutil 

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Lanunch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

In [ ]:
# 通过REST api访问ollama
import urllib.request

def query_model(
        prompt,
        model="llama3",
        url="http://localhost:11434/api/chat",
        role="user"
):
    # 构造请求数据
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {
            "seed": 123,
            "temperature": 1.,
            "top_p": 1,
            "num_ctx": 2048
        }
    }

    # dict转化json并编码
    payload = json.dumps(data).encode("utf-8")
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # 解析返回结果
    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]
    
    return response_data

model = "llama3"
result = query_model("What do Llamas eat?", model)
print(result)

## 提取指令数据

In [ ]:
def extract_instruction(text):
    for content in text.split("\n"):
        if content:
            return content.strip()

In [ ]:
# 使用一个特殊的query来提取指令
query = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query, role="assistant")
instruction = extract_instruction(result)
print(instruction)

In [ ]:
# 使用指令生成响应(回答)
response = query_model(instruction, role="user")
print(response)

In [ ]:
# 生成数据集
from tqdm import tqdm 

dataset_size = 5
dataset = []

for i in tqdm(range(dataset_size)):
    result = query_model(query, role="assistant")
    instruction = extract_instruction(result)
    response = query_model(instruction, role="user")
    entry = {
        "instruction": instruction,
        "output": response
    }
    dataset.append(entry)

In [ ]:
with open("instruction-data-llama3-7b.json", "w") as file:
    json.dump(dataset, file, indent=4)

In [ ]:
!cat instruction-data-llama3-7b.json